In [ ]:
!pip install keras-tcn

In [3]:
from tcn import TCN
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy import stats
import warnings
from google.colab import files
warnings.filterwarnings("ignore")
import keras

In [4]:
RUTA = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/prediction/tcn/CNN - ET0/'
RUTA2 = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/data_processing/data/'

import importlib.util

def module_from_file(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module
# Equivalencia a from CNN import Ensemble_estruturado as es
es = module_from_file("es",RUTA+"CNN/Ensemble_estruturado.py")

# Equivalencia a from CNN import Ensemble_estruturado as es
basic = module_from_file("basic",RUTA+"CNN/basic.py")
eCNN = module_from_file("eCNN",RUTA+"CNN/Ensemble_CNN.py")
tcnn = module_from_file("tcnn",RUTA+"CNN/tcn-prediction.py")

In [5]:
def get_search_dataset():
    # df1 = pd.read_csv('https://raw.githubusercontent.com/PatriciaLucas/Evapotranspiracao/master/ETo_setelagoas.csv', sep=',')
    # df1 = pd.read_csv(RUTA2+'Eto-Pudahuel.csv', sep=',')
    df1 = pd.read_csv(RUTA2+'Eto-Rinconada.csv', sep=',')
    df1 = df1.set_index(df1['Data'])
    df1.index = pd.to_datetime(df1.index,format="%d-%m-%Y")
    df1 = df1[:'2022-06-01']
    series = df1.iloc[:,1]
    # train = series[:6939]
    # test = series[8035:] #ano de teste: 2015
    train = series[:-365]
    test = series[-365:]
    return train, test

train, test = get_search_dataset()


In [6]:
TYPE_MODEL = 'CNN3'
num = 6
savename = RUTA+'E'+TYPE_MODEL+'_MODELS/Comparison/model_'

In [7]:
ini = 60
for j in range(40,ini):
  train, test = get_search_dataset()
  eCNN = module_from_file("eCNN",RUTA+"CNN/Ensemble_CNN.py")
  model = eCNN.Ensemble_CNN(type_models = TYPE_MODEL, number = num) # Creating the ensemble with 6 CNNx models
  model.generate_models()                                           # Generates CNN models
  print(model.get_maxlag())
  if model.get_maxlag()>=215:
    print('...')
    continue  
  model.fit(data = train.values)

  try:
  # Point Forecast
    yhat, y_test,_ = model.point_forecast(data = test.values, forecast_horizon=7)
    tcnn.saveModels(savename+str(j)+'-',model.trained_models)
    rmse, mae, mape, r2 = model.metrics(yhat = yhat[-105:], y_test=y_test[-105:], forecast_horizon=7)
    model_metrics = pd.DataFrame({'RMSE':rmse,'MAE':mae,'MAPE':mape,'R2':r2})
    model_metrics.to_csv(savename+str(j)+'-Metrics.csv')
    model_genotype = pd.DataFrame(model.models)
    model_genotype.to_csv(savename+str(j)+'-Genotype.csv')
  except:
    print('Error de cantidad de datos, pasando a la siguiente iteración')
    ini = ini+1
    continue

101
63
153
464
...
137
74
142
116
52
263
...
165
55
347
...
82
217
...
277
...
298
...
165
51
82
